<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/ethosbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset.git
!pip install -qq transformers

Cloning into 'Ethos-Hate-Speech-Dataset'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 184 (delta 101), reused 107 (delta 46), pack-reused 0
Receiving objects: 100% (184/184), 12.63 MiB | 7.81 MiB/s, done.
Resolving deltas: 100% (101/101), done.
     |████████████████████████████████| 1.8MB 5.2MB/s 
     |████████████████████████████████| 890kB 49.3MB/s 
     |████████████████████████████████| 3.2MB 48.5MB/s 


In [2]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torchtext import data
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [3]:
data_path='Ethos-Hate-Speech-Dataset/ethos/ethos_data/Ethos_Dataset_Binary.csv'
df = pd.read_csv(data_path, delimiter=';')
df.head()

,comment,isHate
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0


In [4]:
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer

def clean_comment(text, stops=False, stemming=False):
  text = str(text)
  text = text.lower().split()
  text = " ".join(text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"don't", "do not ", text)
  text = re.sub(r"aren't", "are not ", text)
  text = re.sub(r"isn't", "is not ", text)
  text = re.sub(r"%", " percent ", text)
  text = re.sub(r"that's", "that is ", text)
  text = re.sub(r"doesn't", "does not ", text)
  text = re.sub(r"he's", "he is ", text)
  text = re.sub(r"she's", "she is ", text)
  text = re.sub(r"it's", "it is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
  text = re.sub(r",", " ", text)
  text = re.sub(r"\.", " ", text)
  text = re.sub(r"!", " ! ", text)
  text = re.sub(r"\/", " ", text)
  text = re.sub(r"\^", " ^ ", text)
  text = re.sub(r"\+", " + ", text)
  text = re.sub(r"\-", " - ", text)
  text = re.sub(r"\=", " = ", text)
  text = re.sub(r"'", " ", text)
  text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
  text = re.sub(r":", " : ", text)
  text = re.sub(r" u s ", " american ", text)
  text = re.sub(r"\0s", "0", text)
  text = re.sub(r" 9 11 ", "911", text)
  text = re.sub(r"e - mail", "email", text)
  text = re.sub(r"j k", "jk", text)
  text = re.sub(r"\s{2,}", " ", text)
  text = text.lower().split()
  text = [w for w in text if len(w) >= 2]
  # if stemming:
  #   englishStemmer = SnowballStemmer("english")
  #   text = [englishStemmer.stem(w) for w in text]
  # if stops:
  #   text = [w for w in text if w not in stopwords.words('english')]
  text = " ".join(text)
  return text

In [5]:
clean_data=[]
for txt in df.comment:
  clean_data.append(clean_comment(txt, stops=True, stemming=True))
df['tweet']=clean_data

In [6]:
def to_category(score):
  if score <= 0.3:
    return 'NOT'
  elif score>0.7:
    return 'OFF'

  

df['subtask_a']=df.isHate.apply(to_category)
df.dropna(axis=0, how='any', inplace=True)
df.head()
df.info

<bound method DataFrame.info of                                                comment  ...  subtask_a
0            You should know women's sports are a joke  ...        OFF
1      You look like Sloth with deeper Down’s syndrome  ...        OFF
2    You look like Russian and speak like Indian. B...  ...        OFF
3                 Women deserve to be abused, I guess.  ...        OFF
4    Women are made for making babies and cooking d...  ...        OFF
..                                                 ...  ...        ...
993   From the midnight sun where the hot springs blow  ...        NOT
994                        Don't say I'm not your type  ...        NOT
995   And therefore never send to know for whom the...  ...        NOT
996                      And I can't stand another day  ...        NOT
997   All values, unless otherwise stated, are in U...  ...        NOT

[768 rows x 4 columns]>

In [7]:
label_dict = {'OFF':1, 'NOT':0}
df['a_label']=df.subtask_a.replace(label_dict)
df

,comment,isHate,tweet,subtask_a,a_label
0,You should know women's sports are a joke,1.0,you should know women sports are joke,OFF,1
1,You look like Sloth with deeper Down’s syndrome,1.0,you look like sloth with deeper down syndrome,OFF,1
2,You look like Russian and speak like Indian. B...,1.0,you look like russian and speak like indian bo...,OFF,1
3,"Women deserve to be abused, I guess.",1.0,women deserve to be abused guess,OFF,1
4,Women are made for making babies and cooking d...,1.0,women are made for making babies and cooking d...,OFF,1
...,...,...,...,...,...
993,From the midnight sun where the hot springs blow,0.0,from the midnight sun where the hot springs blow,NOT,0
994,Don't say I'm not your type,0.0,do not say am not your type,NOT,0
995,And therefore never send to know for whom the...,0.0,and therefore never send to know for whom the ...,NOT,0
996,And I can't stand another day,0.0,and ca not stand another day,NOT,0


In [8]:
RANDOM_SEED = 42
df_train, df_test = train_test_split(
  df,
  test_size=0.3,
  random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)

In [9]:
df_train

,comment,isHate,tweet,subtask_a,a_label
564,People get views of a religion from other peop...,0.166667,people get views of religion from other people...,NOT,0
139,"Deport all niggers, spics, chinks and other un...",1.000000,deport all niggers spics chinks and other unde...,OFF,1
715,This guy gets my nerves!!!! I'm a Christian w...,0.000000,this guy gets my nerves am christian who is tr...,NOT,0
777,put the speed on 1.5 its so good!,0.000000,put the speed on its so good,NOT,0
18,Well done Slovakia and god bless. DO NOT build...,1.000000,well done slovakia and god bless do not build ...,OFF,1
...,...,...,...,...,...
71,Mentally disabled people are fucking scary. Th...,1.000000,mentally disabled people are fucking scary the...,OFF,1
106,I fucking hate gay and lesbians they are all a...,1.000000,fucking hate gay and lesbians they are all ani...,OFF,1
270,Aren’t you ashamed about your body colour?,0.833333,aren you ashamed about your body colour,OFF,1
665,Who gives a damn we have the ability an the fr...,0.000000,who gives damn we have the ability an the free...,NOT,0


In [10]:
print(len(df_train))
print(len(df_val))
print(len(df_test))

537
115
116


In [11]:
df['a_label']=df.subtask_a.replace(label_dict)

df

,comment,isHate,tweet,subtask_a,a_label
0,You should know women's sports are a joke,1.0,you should know women sports are joke,OFF,1
1,You look like Sloth with deeper Down’s syndrome,1.0,you look like sloth with deeper down syndrome,OFF,1
2,You look like Russian and speak like Indian. B...,1.0,you look like russian and speak like indian bo...,OFF,1
3,"Women deserve to be abused, I guess.",1.0,women deserve to be abused guess,OFF,1
4,Women are made for making babies and cooking d...,1.0,women are made for making babies and cooking d...,OFF,1
...,...,...,...,...,...
993,From the midnight sun where the hot springs blow,0.0,from the midnight sun where the hot springs blow,NOT,0
994,Don't say I'm not your type,0.0,do not say am not your type,NOT,0
995,And therefore never send to know for whom the...,0.0,and therefore never send to know for whom the ...,NOT,0
996,And I can't stand another day,0.0,and ca not stand another day,NOT,0


In [12]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [13]:

from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [27]:

encoder_data_train=tokenizer.batch_encode_plus(
    df_train.tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_val=tokenizer.batch_encode_plus(
    df_val.tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_test=tokenizer.batch_encode_plus(
    df_test.tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

inputs_ids_train=encoder_data_train['input_ids']
attentions_masks_train=encoder_data_train['attention_mask']
labels_train=torch.tensor(df_train.a_label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(df_val.a_label.values)

inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(df_test.a_label.values)

In [15]:
from torch.utils.data import TensorDataset
dataset_train=TensorDataset(inputs_ids_train,
              attentions_masks_train, labels_train)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=3

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)
dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [17]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-6, 
                  eps=1e-8)
epochs = 16

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [18]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [19]:


def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [21]:
epochs = 16
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5806567777135518
Validation loss: 0.6052843645597116
F1 Score (Weighted): 0.6602443876101165



Epoch 2
Training loss: 0.5824336266384444
Validation loss: 0.6087130621457711
F1 Score (Weighted): 0.6602443876101165



Epoch 3
Training loss: 0.5793968673857897
Validation loss: 0.6054490063434992
F1 Score (Weighted): 0.6602443876101165



Epoch 4
Training loss: 0.5915514583694202
Validation loss: 0.6065578628809024
F1 Score (Weighted): 0.6602443876101165



Epoch 5
Training loss: 0.5870084975684822
Validation loss: 0.6042517515329214
F1 Score (Weighted): 0.6602443876101165


...


Epoch 12
Training loss: 0.5865995033160268
Validation loss: 0.6061119551842029
F1 Score (Weighted): 0.6602443876101165



Epoch 13
Training loss: 0.5870623523629578
Validation loss: 0.6063900796266702
F1 Score (Weighted): 0.6602443876101165



Epoch 14
Training loss: 0.58516267757842
Validation loss: 0.6102215876946082
F1 Score (Weighted): 0.6602443876101165



Epoch 15
Training loss: 0.5852196313149436
Validation loss: 0.6054790432636554
F1 Score (Weighted): 0.6602443876101165



Epoch 16
Training loss: 0.5887676252666132
Validation loss: 0.6054790478486282
F1 Score (Weighted): 0.6602443876101165



In [28]:
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [35]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_4.model', map_location=torch.device('cuda')))
_, predictions, true_test = evaluate(dataloader_test)
accuracy_per_class(predictions, true_test)

Class: NOT
Accuracy: 71/72

Class: OFF
Accuracy: 16/44



In [36]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.71717   0.98611   0.83041        72
           1    0.94118   0.36364   0.52459        44

    accuracy                        0.75000       116
   macro avg    0.82917   0.67487   0.67750       116
weighted avg    0.80214   0.75000   0.71441       116

